<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#First-train" data-toc-modified-id="First-train-1">First train</a></span><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1.1">Goal</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-1.2">Imports</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1.3">Load data</a></span></li></ul></li></ul></div>

# First train

## Goal

The goal is to do the first training using the whole dataset. Later I will move this to script.

## Imports

In [1]:
# Use this to reload changes in python scripts
%load_ext autoreload
%autoreload 2

In [36]:
import os
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras as keras
from kaggle_environments import make
import pandas as pd
from tqdm.notebook import tqdm

from luxai.utils import render_game_in_html, set_random_seed
from luxai.cunet import cunet_model, cunet_luxai_model, config
from luxai.input_features import make_input
from luxai.output_features import (
    create_actions_mask, create_output_features,
    UNIT_ACTIONS_MAP, CITY_ACTIONS_MAP)

In [37]:
plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (30, 5)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Load data

In [33]:
def load_match(filepath, player):
    with open(filepath, 'r') as f:
        match = json.load(f)
    
    board, features, unit_output, city_output = [], [], [], []
    for step in range(len(match) - 1):
        observation = match[step][0]['observation']
        if player:
            observation.update(match[step][player]['observation'])
        actions = match[step+1][player]['action'] # notice the step + 150

        ret = make_input(observation)
        board.append(ret[0])
        features.append(ret[1])
        active_units_to_position, active_cities_to_position, units_to_position = ret[2:]
        unit_actions_mask = create_actions_mask(active_units_to_position, observation)
        city_actions_mask = create_actions_mask(active_cities_to_position, observation)
        unit_actions, city_actions = create_output_features(actions, units_to_position, observation)
        unit_output.append(np.concatenate([unit_actions, unit_actions_mask], axis=-1))
        city_output.append(np.concatenate([city_actions, city_actions_mask], axis=-1))

    board = np.array(board, dtype=np.float32)
    features = np.array(features, dtype=np.float32)
    unit_output = np.array(unit_output, dtype=np.float32)
    city_output = np.array(city_output, dtype=np.float32)
    return board, features, unit_output, city_output

In [7]:
df = pd.read_csv('/mnt/hdd0/Kaggle/luxai/agent_selection.csv')
df.head()

,Id,EpisodeId,Index,Reward,State,SubmissionId,InitialConfidence,InitialScore,UpdatedConfidence,UpdatedScore,FinalScore
0,68011940,26457630,1,30012.0,2,22612130,170.000000,759.302333,155.000000,819.649388,1516.1269
1,68012510,26457915,1,30003.0,2,22612130,95.351479,1011.992042,90.778073,1036.596193,1516.1269
2,68012021,26457671,0,10009.0,2,22612130,155.000000,819.649388,140.000000,877.636674,1516.1269
3,68012912,26458116,1,5.0,2,22612130,73.168928,1159.946114,70.617282,1180.461034,1516.1269
4,68013075,26458198,0,50015.0,2,22612130,68.590395,1195.431920,66.565502,1212.219363,1516.1269


In [38]:
matches = []
for episode_id, player in tqdm(zip(df.EpisodeId, df.Index), total=len(df)):
    filepath = os.path.join('/mnt/hdd0/Kaggle/luxai/matches_20211014/matches', '%i.json' % episode_id)
    matches.append(load_match(filepath, player))

  0%|          | 0/12791 [00:00<?, ?it/s]

KeyError: 'u_15'